<a href="https://colab.research.google.com/github/megmia40/CovidExaminations/blob/main/Examining_Covid_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Access data from spreadsheets by connecting with google
# Run this code and enter a key to authenticate every session
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1heIQTJ3rS4wXnzrVXYkRIzB8u3zbvdQih5Q-F9dUY74/edit?usp=sharing')

# Importing data into a Pandas data frame
sheet = wb.worksheet('all-states-history') #change this sheet name for a new dataset
data = sheet.get_all_values()

# Storing data in dataframe
df = pd.DataFrame(data)

# Headings
df.columns = df.iloc[0]
df = df.iloc[1:]

# Printing data to check
df.head()

,date,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,inIcuCurrently,negative,negativeIncrease,negativeTestsAntibody,negativeTestsPeopleAntibody,negativeTestsViral,onVentilatorCumulative,onVentilatorCurrently,positive,positiveCasesViral,positiveIncrease,positiveScore,positiveTestsAntibody,positiveTestsAntigen,positiveTestsPeopleAntibody,positiveTestsPeopleAntigen,positiveTestsViral,recovered,totalTestEncountersViral,totalTestEncountersViralIncrease,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
1,2021-01-16,AK,A,229,,0,,1152,1152,67,0,,,1348632,9241,,,1337142,,9,50017,,182,0,,,,,59982,7165,,0,1398649,9423,,,,,,0,1398649,9423
2,2021-01-16,AL,A,6119,5098,89,1021,38763,38763,2772,0,2483,,1686680,6552,,,,1419,,420681,334443,3153,0,,,,,,221961,,0,2021123,8910,,,96429,,2021123,8910,,0
3,2021-01-16,AR,A+,4293,3550,30,743,12686,12686,1292,88,,421,2043157,10884,,,2043157,1348,214,270178,218048,2543,0,,,,61405,,240051,,0,2261205,12593,,,,311571,,0,2261205,12593
4,2021-01-16,AS,N/A,0,,0,,,,,0,,,2140,0,,,,,,0,0,0,0,,,,,,,,0,2140,0,,,,,,0,2140,0
5,2021-01-16,AZ,A+,11248,10115,208,1133,46648,46648,4849,983,,1144,2563182,11054,,,,,821,666901,629995,8715,0,,,,,,90640,,0,5969026,56755,403762,,,,3193177,19272,5969026,56755


In [ ]:
# Extracting the most recent data as of Jan 16th
df.head()
last_day_data = df[0:56]
last_day_data.head(10)

,date,state,dataQualityGrade,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,inIcuCumulative,inIcuCurrently,negative,negativeIncrease,negativeTestsAntibody,negativeTestsPeopleAntibody,negativeTestsViral,onVentilatorCumulative,onVentilatorCurrently,positive,positiveCasesViral,positiveIncrease,positiveScore,positiveTestsAntibody,positiveTestsAntigen,positiveTestsPeopleAntibody,positiveTestsPeopleAntigen,positiveTestsViral,recovered,totalTestEncountersViral,totalTestEncountersViralIncrease,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
1,2021-01-16,AK,A,229,,0,,1152,1152,67,0,,,1348632,9241,,,1337142,,9,50017,,182,0,,,,,59982,7165,,0,1398649,9423,,,,,,0,1398649,9423
2,2021-01-16,AL,A,6119,5098,89,1021,38763,38763,2772,0,2483,,1686680,6552,,,,1419,,420681,334443,3153,0,,,,,,221961,,0,2021123,8910,,,96429,,2021123,8910,,0
3,2021-01-16,AR,A+,4293,3550,30,743,12686,12686,1292,88,,421,2043157,10884,,,2043157,1348,214,270178,218048,2543,0,,,,61405,,240051,,0,2261205,12593,,,,311571,,0,2261205,12593
4,2021-01-16,AS,N/A,0,,0,,,,,0,,,2140,0,,,,,,0,0,0,0,,,,,,,,0,2140,0,,,,,,0,2140,0
5,2021-01-16,AZ,A+,11248,10115,208,1133,46648,46648,4849,983,,1144,2563182,11054,,,,,821,666901,629995,8715,0,,,,,,90640,,0,5969026,56755,403762,,,,3193177,19272,5969026,56755
6,2021-01-16,CA,B,32960,,669,,,,21579,0,,4846,34902009,312097,,,,,,2900246,2900246,40622,0,,,,,,,,0,37802255,352719,,,,,,0,37802255,352719
7,2021-01-16,CO,A,5363,4679,20,684,20664,20664,911,37,,,1932033,6809,276206,,,,,373483,356169,2025,0,38074,,,,,19762,4973741,36638,4973741,36638,316534,,,,2288202,8588,,0
8,2021-01-16,CT,B,6594,5355,0,1239,12257,12257,1098,0,,,4657514,0,,,,,,223422,210193,0,0,,,,,,9800,,0,4880936,0,,186893,,,,0,4867707,0
9,2021-01-16,DC,A+,847,,5,,,,280,0,,67,966955,13300,,,,,29,33537,,397,0,,,,,,23319,1000492,13697,1000492,13697,,,,,379319,2461,,0
10,2021-01-16,DE,A+,1016,910,14,106,,,441,0,,54,481791,1525,,,,,,69372,66226,645,0,,,,,71272,18851,1108131,6641,1108131,6641,,,,,551163,2170,,0


In [ ]:
# Subsetting the data to only include : "states","dataQualityGrade","death",
#"hospitalized","negative","positive","recovered","totalTestResults"

dfs = last_day_data[["state","dataQualityGrade","death","hospitalized","negative","positive","recovered","totalTestResults"]]

dfs.head()

,state,dataQualityGrade,death,hospitalized,negative,positive,recovered,totalTestResults
1,AK,A,229,1152,1348632,50017,7165,1398649
2,AL,A,6119,38763,1686680,420681,221961,2021123
3,AR,A+,4293,12686,2043157,270178,240051,2261205
4,AS,N/A,0,,2140,0,,2140
5,AZ,A+,11248,46648,2563182,666901,90640,5969026


In [50]:
#Remove data of states (rows) with the data quality grade, "D" 
#Store the rest of data in a new data frame

goodData = dfs[dfs.dataQualityGrade != 'D']
goodData = goodData.reset_index(drop=True)

goodData = dfs[dfs.negative != '']
goodData = goodData.reset_index(drop=True)

goodData["death"] = goodData["death"].astype(int)
goodData["negative"] = goodData["negative"].astype(int)
goodData["positive"] = goodData["positive"].astype(int)
goodData["totalTestResults"] = goodData["totalTestResults"].astype(int)

goodData.describe()


,death,negative,positive,totalTestResults
count,55.000000,5.500000e+01,5.500000e+01,5.500000e+01
mean,7027.363636,3.884410e+06,4.257738e+05,5.075726e+06
std,8385.227326,6.216741e+06,5.222350e+05,6.854085e+06
min,0.000000,2.140000e+03,0.000000e+00,2.140000e+03
25%,1544.000000,7.439245e+05,1.000785e+05,1.176639e+06
50%,4293.000000,1.977712e+06,2.859700e+05,2.919566e+06
75%,8217.500000,4.067352e+06,5.740300e+05,5.928363e+06
max,32960.000000,3.490201e+07,2.900246e+06,3.780226e+07


In [51]:
#Finding states with cumulative death counts greater than the average 
average_death = goodData["death"].astype(float).mean()
isHighDeath = goodData['death'].astype(float) > average_death

high_death = goodData[isHighDeath]
print(high_death[['state','death']])

0  state  death
4     AZ  11248
5     CA  32960
10    FL  24380
11    GA  12291
15    IL  20020
16    IN   9287
19    LA   8080
20    MA  13583
23    MI  14669
29    NC   8016
33    NJ  20414
36    NY  32725
37    OH  10135
40    PA  19188
45    TN   8355
46    TX  31831


In [66]:
# Creating a new column called "positivity_rate" in the dataframe "goodData" which is the ration of the values in the column "positive" by those in "goodData"
goodData['positivity_rate'] = goodData['positive'] / goodData['totalTestResults']
goodData.head()

# A conditional which says that if goodData has a positivity rate between 0 and 5%, there regulations in place are working.
for n, row_value in goodData['positivity_rate'].iteritems():
      if goodData['positivity_rate'][n] > 0.00 and goodData['positivity_rate'][n] < 0.05: 
          print(goodData['state'][n], "has a positive covid testing rate of", round(goodData['positivity_rate'][n], 3), "and has good regulations in place already. ")
      if goodData['positivity_rate'][n] > 0.05 and goodData['positivity_rate'][n] < 0.2: 
          print(goodData['state'][n], "has a positive covid testing rate of", round(goodData['positivity_rate'][n], 3), "and needs better regulations. ")
      if goodData['positivity_rate'][n] > 0.2 and goodData['positivity_rate'][n] < 1.0: 
          print(goodData['state'][n],"has a positive covid testing rate of", round(goodData['positivity_rate'][n], 3),"and desperately needs better regulations and regulation enforcement.")

AK has a positive covid testing rate of 0.036 and has good regulations in place already. 
AL has a positive covid testing rate of 0.208 and desperately needs better regulations and regulation enforcement.
AR has a positive covid testing rate of 0.119 and needs better regulations. 
AZ has a positive covid testing rate of 0.112 and needs better regulations. 
CA has a positive covid testing rate of 0.077 and needs better regulations. 
CO has a positive covid testing rate of 0.075 and needs better regulations. 
CT has a positive covid testing rate of 0.046 and has good regulations in place already. 
DC has a positive covid testing rate of 0.034 and has good regulations in place already. 
DE has a positive covid testing rate of 0.063 and needs better regulations. 
FL has a positive covid testing rate of 0.087 and needs better regulations. 
GA has a positive covid testing rate of 0.115 and needs better regulations. 
GU has a positive covid testing rate of 0.074 and needs better regulations. 